In [1]:
import pandas as pd
import json

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.layers import Dropout

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


# Selectd Frames

In [2]:
selected_frames = json.load(open('data/selected_frames.json'))
selected_frames[0]

{'content': '七年前，年僅九歲及八歲的板橋市某國小劉姓姊妹，遭陳昆明殘忍殺害。案發後，他企圖以精神病脫罪，但更一審送鑑定結果是「無精神病」。無奈的是，原本獲判無期徒刑的陳嫌，兩度更審讓刑期縮減為十二年，減刑出獄的結果，卻是葬送另一條寶貴生命。\n九十二年四月十六日，劉姓姊妹放學後，從此人間蒸發。警方成立「○四一六專案小組」，但案情始終陷入膠著，直到死者父親回憶起諸多蛛絲馬跡，發現九十年間因店面樓上整修，臨時工陳昆明出入近兩個月，從而與他的小孩混得很熟，直到陳當兵才失去聯絡。警方靠著這條線索，加上鄰居在案發前見到陳昆明在劉家附近徘徊，鎖定他涉有重嫌。\n陳昆明案發前在劉家外等候兩姊妹，相約十六日放學後一同前往大賣場，兩姊妹上鉤後，他將兩人載往華中橋墩下小木屋，先騙姊姊喝下摻有ＦＭ２的飲料，待藥效發作，動手掐死姊姊；陳嫌接著再掐妹妹頸部，但妹妹踢打反抗、哀嚎，陳昆明隨手持磚塊重擊她頭部致死，事後將兩女童屍體丟入新店溪。\n女童雙屍案偵結後，檢方依殺人罪將陳昆明提起公訴並具體求處極刑，一、二審均判處無期徒刑，但上訴高院之後，陳昆明的委任律師請求再送專門治療精神病的桃園療養院鑑定，但高院未予理會並維持原判。案經上訴，最高法院認為，陳昆明的精神狀態確有詳查必要，乃發回更審。\n不料，高院更一審將陳昆明送桃園療養院鑑定，結果卻是「陳昆明並無精神病」，雖維持原判，但後續更審的結果，陳昆明先改判十六年，再改判十二年，最後搭上減刑的順風車，在去年七月出獄。\n原本應與世隔絕的陳昆明，受惠國內司法制度減刑出獄，但他心中的惡魔卻伺機而動。經過一年的計畫，從假藉應徵手段到分租單身公寓，陳昆明預謀犯案的慾念似乎從未間斷，而現行司法制度間接造成無辜的林女枉死，未來應更慎重斟酌量刑。\n(中國時報)\n',
 'from': '顏玉龍／新聞幕後',
 'law': ['殺人罪'],
 'pos_content': [['七', 'Neu'],
  ['年前', 'Nd'],
  ['，', 'COMMACATEGORY'],
  ['年', 'Na'],
  ['僅', 'Da'],
  ['九', 'Neu'],
  ['歲', 'Nf'],
  ['及', 'Caa'],
  ['八', 'Neu'],
  ['歲', 'Nf'],
  ['的', 'DE'],
  ['板橋市'

# All Possible Laws

In [3]:
occurrence = pd.read_csv('data/occurrence.csv', header=None)
laws = list(occurrence[0].as_matrix())
laws

['殺人罪',
 '公共危險罪',
 '竊盜罪',
 '傷害罪',
 '民法',
 '銀行法',
 '妨害性自主罪',
 '社會秩序維護法',
 '毒品危害防制條例',
 '賭博罪',
 '藥事法',
 '刑事訴訟法',
 '證券交易法',
 '會計法',
 '妨害風化罪',
 '動物保護法',
 '槍砲彈藥刀械管制條例',
 '商業會計法',
 '政府採購法',
 '侵占罪',
 '大學法',
 '公司法',
 '妨害公務罪',
 '廢棄物清理法',
 '道路交通管理處罰條例',
 '著作權法',
 '醫師法',
 '法官法',
 '人口販運防制法',
 '妨害自由罪',
 '野生動物保育法',
 '森林法',
 '洗錢防制法',
 '組織犯罪防制條例',
 '商標法',
 '稅捐稽徵法',
 '性騷擾防治法',
 '公平交易法',
 '遺棄罪',
 '就業服務法',
 '刑事妥速審判法',
 '建築法',
 '公務員服務法',
 '證人保護法',
 '入出國及移民法',
 '贓物罪',
 '國家機密保護法',
 '醫療法',
 '陸海空軍刑法',
 '保險法',
 '國家安全法',
 '食品安全衛生管理法',
 '公務員懲戒法',
 '妨害電腦使用罪',
 '兒童及少年福利與權益保障法',
 '少年事件處理法',
 '少年福利法',
 '個人資料保護法',
 '妨害秘密罪',
 '律師法',
 '法院組織法',
 '監察法',
 '軍事審判法',
 '集會遊行法',
 '外患罪',
 '消防法',
 '人類免疫缺乏病毒傳染防治及感染者權益保障條例',
 '勞動基準法',
 '土地法',
 '性侵害犯罪防治法',
 '漁業法',
 '偽造有價證券罪',
 '教師法',
 '民用航空法',
 '菸酒管理法',
 '公路法',
 '性別工作平等法',
 '懲治盜匪條例',
 '行政執行法',
 '通訊保障及監察法',
 '國籍法',
 '地方制度法',
 '大眾捷運法',
 '懲治走私條例',
 '水土保持法',
 '瀆職罪',
 '護照條例',
 '鐵路法',
 '電信法',
 '健康食品管理法',
 '刑事補償法',
 '所得稅法',
 '文化資產保存法',
 '消費者保護法',
 '犯罪被害人保護法',
 '緊急醫療救護法',
 '脫逃罪',
 '行政程序法',

# Parameters

In [4]:
n_classes = len(laws)
n_svd = 300

# Generate Training Data

In [5]:
train_x, train_y = [], []

for frame in selected_frames:
    # Select articles with only one law as our training data
    if len(frame['law']) == 1:
        law_ind = laws.index(frame['law'][0])
        assert(law_ind >= 0)
        seg_content = ' '.join(list(zip(*frame['pos_content']))[0])
        seg_content = seg_content.replace(frame['law'][0], '')
        assert(frame['law'][0] not in seg_content)
        train_x.append(seg_content)
        train_y.append(law_ind)
    
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(train_x)
svd = TruncatedSVD(n_svd)
normalizer = Normalizer()
lsa = make_pipeline(svd, normalizer)

train_x = lsa.fit_transform(vectors)
train_y = to_categorical(train_y, n_classes)

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.25, random_state=42)

# A Garbage Model

In [6]:
def get_garbage_model():
    model = Sequential()
    model.add(Dense(n_svd // 2, activation='relu', input_dim=n_svd))
    model.add(Dropout(0.3))
    model.add(Dense(n_svd // 4, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [7]:
model = get_garbage_model()
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x ,train_y, epochs=50, validation_data=(valid_x, valid_y))


Train on 3016 samples, validate on 1006 samples
Epoch 1/50
3016/3016 [==============================] - 0s - loss: 4.9179 - acc: 0.1379 - val_loss: 4.0590 - val_acc: 0.1511
Epoch 2/50
3016/3016 [==============================] - 0s - loss: 3.8284 - acc: 0.1631 - val_loss: 3.7372 - val_acc: 0.2256
Epoch 3/50
3016/3016 [==============================] - 0s - loss: 3.3962 - acc: 0.2941 - val_loss: 3.3378 - val_acc: 0.3559
Epoch 4/50
3016/3016 [==============================] - 0s - loss: 2.9628 - acc: 0.4194 - val_loss: 3.0453 - val_acc: 0.4274
Epoch 5/50
3016/3016 [==============================] - 0s - loss: 2.6189 - acc: 0.4788 - val_loss: 2.8444 - val_acc: 0.4573
Epoch 6/50
3016/3016 [==============================] - 0s - loss: 2.3514 - acc: 0.5269 - val_loss: 2.7102 - val_acc: 0.4911
Epoch 7/50
3016/3016 [==============================] - 0s - loss: 2.1305 - acc: 0.5680 - val_loss: 2.5795 - val_acc: 0.5129
Epoch 8/50
3016/3016 [==============================] - 0s - loss: 1.9490 - a